In [0]:
# UNSW-NB15
# This file handles the processing of the UNSW data
# This andles the normalization and data processing 
# of taking particular information from the UNSW
# processed excel files 

In [0]:
# Initialize drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
# Move to drive 
%cd drive
%cd 'My Drive'
%cd Oxford-Thesis

/content/drive
/content/drive/My Drive
/content/drive/My Drive/Oxford-Thesis


In [0]:
import argparse
import os
import sys
import pickle
import re 
import numpy as np
import csv
import string

In [0]:
## Get diferent types of flows
flows = []
benign_flows = []
scan_flows =[]
dos_flows = []
exp_flows =[]
analy_flows = []
fuzz_flows = []
back_flows = []
gen_flows =[]
shell_flows = []
worm_flows = []

train_benign_flows = []
train_scan_flows =[]
test_benign_flows = []
test_scan_flows =[]

proto = [] ## Protocol
state = [] ## State
service = [] ## Service 
all_ports = [0]*65536 ## Possible Ports

In [0]:
## Check if value is actually valid in the excel file
def valid_value(s):
  if s == '-':
    return False
  return True

## Check if the value is a hex value
def is_hex(s):
  if (len(s) > 2):
    if s[0] == '0' and s[1] =='x':
      return True
  return False

## Change the hex value to a decimal vlaue
def hex_to_dec(s):
  return int(s[2:], 16)


In [0]:
## Read a given UNSW file
def read_file(file,service):
  with open(file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
      if int(row[48]) == 0 and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))   
        benign_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin])
      elif 'Recon' in row[47] and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))  
        scan_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin])
      elif 'DoS' in row[47] and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))    
        dos_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin])
      elif 'Expl' in row[47] and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))    
        exp_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin])
      elif 'Fuzz' in row[47] and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))    
        fuzz_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin])  
      elif 'Analy' in row[47] and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))    
        analy_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin]) 
      elif 'Back' in row[47] and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))    
        back_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin])    
      elif 'Gen' in row[47] and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))    
        gen_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin]) 
      elif 'Shell' in row[47] and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))    
        shell_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin])   
      elif 'Worm' in row[47] and service in row[13]:
        ports = []
        new_flow_cont = []
        new_flow_cat = []
        new_flow_bin = []
        for index in range(0,len(row)):
          if index == 0: ## src ip 
            continue
          elif index == 2: ## dst ip
            continue 
          elif index == 1:  ## src port
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 3: ## dst port 
            if is_hex(row[index]): 
              if hex_to_dec(row[index]) <65536:
                all_ports[hex_to_dec(row[index])]+=1
                ports.append(hex_to_dec(row[index]))
              else:
                ports.append(int(0))
            elif valid_value(row[index]):
              if int(row[index]) <65536:
                all_ports[int(row[index])]+=1
                ports.append(int(row[index]))
              else:
                ports.append(int(0))
            else:
              ports.append(int(0)) 
          elif index == 4:
            if row[index] not in proto:
              proto.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 5:
            if row[index] not in state:
              state.append(row[index])
            new_flow_cat.append(row[index])
          elif index == 35:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index == 38:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_bin.append(int(row[index]))
            else:
              new_flow_bin.append(int(0))
          elif index >= 6 and index <= 12:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 14 and index <= 27:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 28 and index <= 34:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 36 and index <= 37:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))
          elif index >= 39 and index <= 46:
            if row[index] != "-" and row[index] != ''and row[index] != ' ':
              new_flow_cont.append(float(row[index]))
            else:
              new_flow_cont.append(float(0))    
        worm_flows.append([ports,new_flow_cont,new_flow_cat,new_flow_bin])                

In [0]:
read_file('./datasets/UNSW-NB15/UNSW-NB15_1.csv','http')


184


In [0]:
read_file('./datasets/UNSW-NB15/UNSW-NB15_2.csv','http')


586


In [0]:
read_file('./datasets/UNSW-NB15/UNSW-NB15_3.csv','http')


1264


In [0]:
read_file('./datasets/UNSW-NB15/UNSW-NB15_4.csv','http')


1709


In [0]:
## Get the largest ports
largest_ports = []
while len(largest_ports) < 64:
  large_value = max(all_ports)
  large_index = all_ports.index(large_value)
  largest_ports.append(large_index)
  all_ports[large_index] = 0 

64
1
4
0


In [0]:
benign_label = [[1,0,0,0,0,0,0,0,0,0]]
scan_label = [[0,1,0,0,0,0,0,0,0,0]]
fuzz_label = [[0,0,1,0,0,0,0,0,0,0]]
analy_label = [[0,0,0,1,0,0,0,0,0,0]]
back_label = [[0,0,0,0,1,0,0,0,0,0]]
dos_label = [[0,0,0,0,0,1,0,0,0,0]]
exp_label = [[0,0,0,0,0,0,1,0,0,0]]
gen_label = [[0,0,0,0,0,0,0,1,0,0]]
shell_label = [[0,0,0,0,0,0,0,0,1,0]]
worm_label = [[0,0,0,0,0,0,0,0,0,1]]

flows = benign_flows+ scan_flows+fuzz_flows +analy_flows+back_flows+dos_flows+exp_flows+gen_flows+ shell_flows+worm_flows
labels = benign_label*len(benign_flows) + scan_label*len(scan_flows)+fuzz_label*len(fuzz_flows)
labels = labels+analy_label*len(analy_flows)+back_label*len(back_flows)+dos_label*len(dos_flows)
labels = labels+exp_label*len(exp_flows)+gen_label*len(gen_flows)+shell_label*len(shell_flows)
labels = labels+worm_label*len(worm_flows)
                                                            

1709
2073
187426
2073
1087
558
92
1709
11481
1699
0
148


In [0]:
## Process the continuous and the port features
processed_flows = []
while len(flows) > 0:
  for i in range(0,len(flows)):
    binaries = []
    conts = []
    for index in range(0,len(flows[i])):
      if index == 0:
        for port in flows[i][index]:
          one_hot = [0]*len(largest_ports)
          if port in largest_ports:
            case = largest_ports.index(port)  
          else:
            case = 0 
          one_hot[case] +=1
          for one in one_hot:
            binaries.append(one)
      elif index == 1:
        for cont in flows[i][index]:
          conts.append(cont)    
      elif index == 2:
        for index2 in range(0,len(flows[i][index])):
          if index2 == 0:
            one_hot = [0]*len(proto)
            case = proto.index(flows[i][index][index2])  
            one_hot[case] +=1
            for one in one_hot:
              binaries.append(one)
          elif index2 == 1:
            one_hot = [0]*len(state)
            case = state.index(flows[i][index][index2])  
            one_hot[case] +=1
            for one in one_hot:
              binaries.append(one)
          elif index2 == 2:
            one_hot = [0]*len(service)
            case = service.index(flows[i][index][index2])  
            one_hot[case] +=1
            for one in one_hot:
              binaries.append(one)
      elif index == 3:
        for index2 in range(0,len(flows[i][index])):
          binaries.append(flows[i][index][index2])
    processed_flows.append([binaries,conts])
    del flows[i]
    break
  

In [0]:
import numpy as np
data = np.array(processed_flows)
del processed_flows
print(data.shape)
binaries = np.array(data[:,0])
conts = np.array(data[:,1])
del data

(206273, 2)


In [0]:
## Process the continuous features
processed_conts = []
for i in conts:
  processed_conts.append(i)
processed_conts = np.array(processed_conts)

processed_bins = []
for i in binaries:
  processed_bins.append(i)
processed_bins = np.array(processed_bins)

print(processed_conts.shape)
for i in range(0,38):
  max_value = np.max(processed_conts[:,i])
  min_value = np.min(processed_conts[:,i])
  processed_conts[:,i] = ((processed_conts[:,i] - min_value)/(max_value-min_value +.00000000001))

(206273, 38)


In [0]:
## Dump the continuous flows for later analysis 
with open('./embedding/UNSW-NB15/continuous1HTTPALL', 'wb') as fp:
  pickle.dump(processed_conts,fp)
  fp.close()
del processed_conts
gc.collect()

22

In [0]:
## Dump the binar flows for later analysis 
with open('./embedding/UNSW-NB15/binaries1HTTPALL', 'wb') as fp:
  pickle.dump(processed_bins,fp)
  fp.close()
